自动文摘的目的是通过对原文本进行压缩、提炼，为用户供简明扼要的文字描述。自动文摘是一个信息压缩过程，将输入的一篇或多篇文档内容总结为一段简要描述，该过程不可避免有信息损失，但是要求保留尽可能多的重要信息，自动文摘也是自然语言生成领域中一个重要任务。
下面我们以文本摘要任务为例，展示孟子预训练模型在下游任务上微调的流程，整体流程可以分为4部分：

- 数据加载
- 数据预处理
- 模型训练
- 模型推理
- 评测

下面我们以中文科学文献数据（CSL）文本摘要数据为例进行演示，数据下载地址：https://github.com/CLUEbenchmark/CLGE

下载的原始数据：训练集(3,000)，验证集(500)，测试集(500)，但测试集没有摘要标注结果，所以这里我们简单地把验证集当作测试集，从训练集中划出500条作为开发集。

## 依赖环境
代码使用以下环境运行
- transformers==4.12.5
- sentencepiece==0.1.95
- rouge==1.0.1
- nltk==3.6.5

## 1. 数据加载

CSL数据以json的形式存储，通过如下方式可以将数据加载进内存。

In [1]:
import json
import time
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
from tqdm import tqdm
import torch
import random

In [2]:
def read_json(input_file: str) -> list:
    '''
    读取json文件，每行是一个json字段

    Args:
        input_file:文件名

    Returns:
        lines
    '''
    with open(input_file, 'r') as f:
        lines = f.readlines()
    return list(map(json.loads, tqdm(lines, desc='Reading...')))

trainset = read_json("csl/csl_title_train.json") 
test = read_json("csl/csl_title_dev.json") 

Reading...: 100%|██████████| 500/500 [00:00<00:00, 221826.95it/s]


下面展示数据集的具体信息：

In [4]:
random.shuffle(trainset)
train = trainset[:2500]
dev = trainset[2500:]
print('训练集大小：%d个训练样本'%(len(train)))
print('开发集大小：%d个训练样本'%(len(dev)))
print('每个训练样本的原始格式如下：\n',train[1])

训练集大小：2500个训练样本
开发集大小：500个训练样本
每个训练样本的原始格式如下：
 {'id': 2364, 'title': '基于语义规则的Web服务发现方法', 'abst': '语义Web服务发现问题研究的核心内容是服务描述与对应的服务发现方法。服务描述分为服务请求描述与服务发布描述,但目前的服务发现方法,并未将请求描述与发布描述分开,以比对服务请求描述与服务发布描述中对应部分作为匹配依据,导致服务请求描述构建困难以及发现结果不够理想。提出以语义规则刻画服务请求描述,以本体构建服务发布描述,进行有效的以语义规则驱动的Web服务发现。对语义规则添加影响因子使得服务匹配精度可以通过匹配度来度量,并按照给定的调节系数来决定最终匹配是否成功。最后以OWL-STCV2测试服务集合进行了对比实验,证实该方法有效地提高了查全率与查准率高,特别是Top-k查准率。'}


### 可以看出每条原始数据包含3个字段，分别是id，title，abst，其中id是唯一标识，abst是文本摘要任务的输入，title是文本摘要任务的输出。

## 2. 数据预处理

数据预处理的目的是将原始数据处理为模型可以接受的输入形式，相当于在原始数据和模型输入之间建立管道。
模型输入，可接受的字段为input_ids、labels，其中input_ids为输入文本的tokenized表示，可以直接通过transformers提供的Tokenizer进行转换；labels为模型期望输出文本的tokenized表示。
通过定义DataCollatorForSeq2Seq数据预处理类，将其传递给data_collator完成上述流程，数据预处理代码如下：

In [56]:
model_path = "Langboat/mengzi-t5-base" # huggingface下载模型

##### 加载预训练模型，包括分词器tokenizer和model。

In [7]:
Mengzi_tokenizer = T5Tokenizer.from_pretrained(model_path)

In [8]:
Mengzi_model = T5ForConditionalGeneration.from_pretrained(model_path)

In [18]:
class Seq2SeqDataset:
    def __init__(self, data):
        self.datas = data

    def __len__(self):
        return len(self.datas)

    def __getitem__(self, index):
        return self.datas[index]

class DataCollatorForSeq2Seq:
    def __init__(self, tokenizer, padding: bool = True, max_length: int = 512):
        self.tokenizer = tokenizer
        #self.model = model
        self.padding = padding
        self.max_length = max_length

    def __call__(self, batch):
        features = self.collator_fn(batch)
        return features


    def preprocess(self, item):
        source = item["abst"]
        target = item["title"]
        return source, target

    def collator_fn(self, batch):
        results = map(self.preprocess, batch)
        inputs, targets = zip(*results)

        input_tensor = self.tokenizer(inputs,
                                      truncation=True,
                                      padding=True,
                                      max_length=self.max_length,
                                      return_tensors="pt",
                                      )

        target_tensor = self.tokenizer(targets,
                                       truncation=True,
                                       padding=True,
                                       max_length=self.max_length,
                                       return_tensors="pt",
                                       )

        input_tensor["labels"] = target_tensor["input_ids"]

        if "token_type_ids" in input_tensor:
            del input_tensor["token_type_ids"]
        return input_tensor

In [19]:
trainset = Seq2SeqDataset(train)
devset = Seq2SeqDataset(dev)

In [20]:
collator = DataCollatorForSeq2Seq(Mengzi_tokenizer)

## 3. 模型训练

训练模型前需要指定模型训练的超参数，包括训练的轮数、学习率和学习率管理策略等等：可以通过实例化TrainingArguments类来，并将其传递给Trainer来传入这些超参数。
然后通过huggingface定义的trainer.train()方法来进行训练。
训练完成后通过trainer.save_model()方法来保存最佳模型。

In [22]:
output_dir = "test" # 模型checkpoint的保存目录
training_args = TrainingArguments(
        num_train_epochs=3,
        per_device_train_batch_size=8,
        logging_steps=10,
        #fp16=True,
        evaluation_strategy="steps",
        eval_steps=100,
        load_best_model_at_end=True,
        learning_rate=1e-5,
        #warmup_steps=100,
        output_dir="test",
        save_total_limit=5,
        lr_scheduler_type='constant',
        gradient_accumulation_steps=1,
        dataloader_num_workers=4)

In [23]:
print('Tarining Arguments ...')
print(training_args)

trainer = Trainer(
    tokenizer=Mengzi_tokenizer,
    model=Mengzi_model,
    args=training_args,
    data_collator=collator,
    train_dataset=trainset,
    eval_dataset=devset
)

trainer.train()
trainer.save_model("test/best") # 保存最好的模型

Tarining Arguments ...
TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=4,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_steps=100,
evaluation_strategy=IntervalStrategy.STEPS,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
greater_is_better=False,
group_by_length=False,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=1e-05,
length_column_name=length,
load_best_model_at_end=True,
local_rank=-1,
log_level=-1,
log_level_replica=-1,
log_on_each_node=True,
logging_dir=test/runs/Nov01_11-42-16_JX-ZY-GPU12,
logging_first_step=False,
logging_steps=10,
logging_strategy=IntervalStrategy.STEPS,
lr_scheduler_type=SchedulerType.CONSTANT,
max_grad_norm=1.0,
max_steps=

***** Running training *****
  Num examples = 2500
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 939
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: egyang (use `wandb login --relogin` to force relogin)


Step,Training Loss,Validation Loss
100,1.781900,1.654125
200,1.669000,1.454569
300,1.399100,1.426801
400,1.294300,1.377249
500,1.362800,1.363519
600,1.497200,1.342906
700,1.297500,1.321924
800,1.294700,1.321687
900,1.111900,1.306872


***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to test/checkpoint-100
Configuration saved in test/checkpoint-100/config.json
Model weights saved in test/checkpoint-100/pytorch_model.bin
tokenizer config file saved in test/checkpoint-100/tokenizer_config.json
Special tokens file saved in test/checkpoint-100/special_tokens_map.json
Copy vocab file to test/checkpoint-100/spiece.model
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to test/checkpoint-200
Configuration saved in test/checkpoint-200/config.json
Model weights saved in test/checkpoint-200/pytorch_model.bin
tokenizer config file saved in test/checkpoint-200/tokenizer_config.json
Special tokens file saved in test/checkpoint-200/special_tokens_map.json
Copy vocab file to test/checkpoint-200/spiece.model
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to test/checkpoint-300
Configuration saved in t

## 4. 模型推理

最佳模型保存在了"test/best"位置，我们可以加载最佳模型并利用其进行摘要生成。
下面是我们利用模型进行推理的一种实现方式，将希望简化的文本tokenized后传入模型，得到经过tokenizer解码后即可获得摘要后的文本。当然，读者也可以利用自己熟悉的方式进行生成。

In [3]:
def preprocess(items):
    inputs = []
    titles = []
    for item in items:
        inputs.append(item["abst"])
        titles.append(item["title"])
    return inputs, titles

In [4]:
best_model = "test/best"
tokenizer = T5Tokenizer.from_pretrained(best_model)
model = T5ForConditionalGeneration.from_pretrained(best_model).cuda()

In [5]:
def predict(sources, batch_size=8):
    _model = model.eval() # 将模型转换为预测模式，使模型内容的droput失效。
    
    kwargs = {"num_beams":4}
    
    outputs = []
    for start in tqdm(range(0, len(sources), batch_size)):
        batch = sources[start:start+batch_size]
        
        input_tensor = tokenizer(batch, return_tensors="pt", truncation=True, padding=True, max_length=512).input_ids.cuda()
        
        outputs.extend(model.generate(input_ids=input_tensor, **kwargs))
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [6]:
inputs, refs = preprocess(test)

In [7]:
inputs[0]

'抽象了一种基于中心的战术应用场景与业务,并将网络编码技术应用于此类场景的实时数据多播业务中。在分析基于中心网络与Many-to-all业务模式特性的基础上,提出了仅在中心节点进行编码操作的传输策略以及相应的贪心算法。分析了网络编码多播策略的理论增益上界,仿真试验表明该贪心算法能够获得与理论相近的性能增益。最后的分析与仿真试验表明,在这种有中心网络的实时数据多播应用中,所提出的多播策略的实时性能要明显优于传统传输策略。'

In [8]:
refs[0]

'网络编码在实时战术数据多播中的应用'

In [9]:
generations = predict(inputs)

100%|██████████| 63/63 [00:25<00:00,  2.49it/s]


In [10]:
generations[0]

'基于中心网络的实时数据多播应用'

## 5. 生成结果的评测

采用自动文摘任务上常用的自动评测指标Rouge-1, Rouge-2, Rouge-L对生成文本的质量进行评测。

In [13]:
from rouge import Rouge 

hypothesis = "the #### transcript is a written version of each day 's cnn student news program use this transcript to help students with reading comprehension and vocabulary use the weekly newsquiz to test your knowledge of storie s you saw on cnn student news"

reference = "this page includes the show transcript use the transcript to help students with reading comprehension and vocabulary at the bottom of the page , comment for a chance to be mentioned on cnn student news . you must be a teacher or a student age # # or older to request a mention on the cnn student news roll call . the weekly newsquiz tests students ' knowledge of even ts in the news"

rouge = Rouge()
scores = rouge.get_scores(hypothesis, reference)


In [14]:
scores

[{'rouge-1': {'r': 0.4583333333333333,
   'p': 0.6285714285714286,
   'f': 0.5301204770503702},
  'rouge-2': {'r': 0.21739130434782608, 'p': 0.375, 'f': 0.2752293531520916},
  'rouge-l': {'r': 0.4166666666666667,
   'p': 0.5714285714285714,
   'f': 0.4819277059660328}}]

In [17]:
from rouge import Rouge
rouge = Rouge()

def rouge_score(candidate, reference):
    text1 = " ".join(list(candidate))
    text2 = " ".join(list(reference))
    score = rouge.get_scores(text1, text2)
    return score

def compute_rouge(preds, refs):
    r1=[]
    r2=[]
    R_L=[]
    for pred, ref in zip(preds, refs):
        scores = rouge_score(pred, ref)
        r1.append(scores[0]["rouge-1"]["f"])
        r2.append(scores[0]["rouge-2"]["f"])
        R_L.append(scores[0]["rouge-l"]["f"])
    return sum(r1)/len(r1), sum(r2)/len(r2), sum(R_L)/len(R_L)

In [18]:
R_1, R_2, R_L = compute_rouge(generations, refs)